Realtime multimodal usecase | extract image,table,text from document | multimodal summrizer

In [4]:
%pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install unstructured-pytesseract
%pip install tesseract-ocr

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cython-3.1.6-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached cython-3.1.6-cp312-cp312-win_amd64.whl (2.7 MB)
Failed to build tesseract-ocr
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for tesseract-ocr (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      C:\Users\Hp\AppData\Local\Temp\pip-build-env-p8qx60ye\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\Hp\AppData\Local\Temp\pip-build-env-p8qx60ye\overlay\Lib\site-packages\setuptools\dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
              (Affected: tesseract-ocr).
      
              By 2026-Mar-

In [1]:
from unstructured.partition.pdf import partition_pdf

d:\multimodel-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_pdf=partition_pdf(
    filename="/multimodel-rag/data/RAG-paper.pdf" , #mandatory
    strategy='hi_res',  #mandatory to use "hi_res" strategy
    extract_images_in_pdf=True, #mandatory to set as True
    extract_image_block_types=["images","Table"], #optional
    extract_image_block_to_payload=False, #optional
    extract_image_block_output_dir="extracted_data"
)

Cannot set gray non-stroke color because /'P1' is an invalid float value


Cannot set gray non-stroke color because /'P1' is an invalid float value
The requested type (images) doesn't match any available type


In [3]:
raw_pdf

In [10]:
Header=[]
footer=[]
Title=[]
NarrativeTexts=[]
Text=[]
ListItem=[]

for element in raw_pdf:
    if "unstructured.documents.elements.Header" in str(type(element)):
        Header.append(str(element))
    elif "unstructured.documents.elements.footer" in str(type(element)):
        footer.append(str(element))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeTexts.append(str(element))
    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))

In [3]:
Text=[]
for element in raw_pdf:
    if "unstructured.documents.elements.NarrativeText" in str(type(element)):
        Text.append(str(element))


In [37]:
Text[6]

'Figure 1: Overview of our approach. We combine a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model (Generator) and ﬁne-tune end-to-end. For query x, we use Maximum Inner Product Search (MIPS) to ﬁnd the top-K documents zi. For ﬁnal prediction y, we treat z as a latent variable and marginalize over seq2seq predictions given different documents.'

In [38]:
img=[]
for element in raw_pdf:
    if "unstructured.documents.elements.Image" in str(type(element)):
        img.append(str(element))


In [39]:
print(len(img))

3


In [5]:
tab=[]
for element in raw_pdf:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tab.append(str(element))
print(len(tab))

4


In [23]:
len(tab)


4

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import  ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import base64
import os
from langchain_core.messages import HumanMessage


In [ ]:
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [24]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

Table summary

In [27]:
prompt_text="""you are an assistant tasked with summarizing tables for retrivel.\
These summaries will be embedded and used to retrieve the raw table element.\
Given a concise summary of the table  that is well optimized for retrivel.Table{element}""" 

In [26]:
prompt=ChatPromptTemplate.from_template(prompt_text)

In [25]:
summarize_chain={"element": lambda x:x}| prompt | model | StrOutputParser()

In [28]:
table_summaries=[]
for i, t in enumerate(tab):
    print(f"Summarizing table {i+1}/{len(tab)}...")
    summary = summarize_chain.invoke(t)
    table_summaries.append(summary)


Summarizing table 1/4...
Summarizing table 2/4...
Summarizing table 3/4...
Summarizing table 4/4...


In [29]:
print(table_summaries[3])

This table compares the performance of various RAG (Retrieval Augmented Generation) models, including RAG-Token-BM25, RAG-Sequence-BM25, RAG-Token-Frozen, RAG-Sequence-Frozen, RAG-Token, and RAG-Sequence, alongside a TableModel. It reports scores across diverse NLP datasets and tasks such as NQ, TQA, WQ, CT, Jeopardy-QGen, MSMarco, and FVR-3, using metrics like Exact Match, B-1, QB-1, R-L B-1, and Label Accuracy.


IMage summary

In [45]:
def encode_image(image_path):
    "getting the base64 string"
    with open(image_path,'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [41]:
def image_summarize(img_base64,prompt):
    """make image summary"""
    #chat=ChatGoogleGenerativeAI(model="gemini-2.5-pro")
    msg=model.invoke(
        [
            HumanMessage(
                content=[
                    {"type":"text","text":prompt},
                    {
                     "type":"image_url",
                     "image_url":{"url":f"data:image/jpeg;base64,{img_base64}"},   
                    },
                ]
            )
        ]
    )
    return msg.content

In [43]:
def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for image
    path:Path to list of .jpg files extracted by unstructured
    """
    #store base64 encoded images
    img_base64_list=[]
    
    #Store image summaries
    image_summarizes=[]
    #prompt
    prompt="""you are an assistant tasked with summarizing image for retrieval
    These summaries will be embedded and used to retrieve the raw image.
    Give a concise summary of the image that is well optimized for retrieval."""

    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path=os.path.join(path,img_file)
                
            base64_image=encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summarizes.append(image_summarize(base64_image,prompt))
            time.sleep(2)
    
    return img_base64_list,image_summarizes

In [46]:
filepath="/multimodel-rag/extracted_data/"
img_base64,image_summaries=generate_img_summaries(filepath)

In [48]:
print(img_base64[1])
print(image_summaries[2])

/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAB5AhADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDtP7Gj/wCglrn/AIO7z/47R/Y0f/QS1z/wd3n/AMdrRoru5I9jz+eXczTo0eP+Qlrn/g7vP/jtN/sdP+glrn/g6u//AI7WmehplHJHsLnl3M/+x0/6CWuf+Dq7/wDjtKujxlgP7S1zr/0Grv8A+O1fpV++PrRyR7Bzy7kX9gW/

Text summary


In [30]:
prompt_text="""you are an assistant tasked with summarizing text for retrivel.\
These summaries will be embedded and used to retrieve the raw text element.\
Given a concise summary of the text  that is well optimized for retrivel.text: {element}""" 

In [31]:
prompt=ChatPromptTemplate.from_template(prompt_text)

In [32]:
summarize_chain={"element": lambda x:x}| prompt | model | StrOutputParser()

In [33]:
len(Text)

61

In [34]:
import time

batch_size = 8  # safe batch size (keeps you below 10 requests/min)
text_summaries = []

for i in range(0, len(Text), batch_size):
    batch = Text[i:i + batch_size]
    print(f"Summarizing batch {i//batch_size + 1}/{(len(Text) + batch_size - 1)//batch_size}...")
    
    try:
        summaries = summarize_chain.batch(batch, {"max_concurrency":2})
        text_summaries.extend(summaries)
    except Exception as e:
        print(f"Error in batch {i//batch_size + 1}: {e}")

    # 🕐 Wait before next batch to avoid hitting rate limit
    time.sleep(65)


Summarizing batch 1/8...
Summarizing batch 2/8...
Summarizing batch 3/8...
Summarizing batch 4/8...
Summarizing batch 5/8...
Summarizing batch 6/8...
Summarizing batch 7/8...
Summarizing batch 8/8...


In [35]:
text_summaries[6]

'This approach combines a pre-trained retriever (Query Encoder + Document Index) with a pre-trained seq2seq model, fine-tuned end-to-end. It uses Maximum Inner Product Search (MIPS) for top-K document retrieval, treats documents as a latent variable, and marginalizes seq2seq predictions for the final output.'

In [49]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.stores import InMemoryStore
from langchain_cohere import CohereEmbeddings

In [60]:
from dotenv import load_dotenv
load_dotenv()
COHERE_API_KEY=os.getenv("COHERE_API_KEY")
# 1️⃣ Initialize Cohere Embeddings
embedding_function = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=COHERE_API_KEY
)

In [58]:
def create_multi_vector_retriever(vectorstore,text,text_summaries,table,table_summaries,image,image_summaries):
    """
    create retriever that indexes summaries,but return raw images and text
    """
    #initialize the storage layer
    store=InMemoryStore()
    id_key="doc_id"
    
    #create the multi-vector retriever 
    retriever=MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )
    
    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever,doc_summaries,doc_contents):
        doc_ids=[str(uuid.uuid4()) for _ in doc_contents]
        
        summary_docs=[
            Document(page_content=s,metadata={id_key:doc_ids[i]})
            for i,s in enumerate(doc_summaries)
        ]
        
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids,doc_contents)))
        
    #add texts,table,image
    #check that text_summary is not empty before adding
    if text_summaries:
        add_documents(retriever,text_summaries,text)
    if table_summaries:
        add_documents(retriever,table_summaries,table)
    if image_summaries:
        add_documents(retriever,image_summaries,image)
    return retriever
            
            
    

In [61]:
vectorstore=Chroma(collection_name="my_rag",embedding_function=embedding_function)
retriever_multivector=create_multi_vector_retriever(vectorstore,Text,text_summaries,tab,table_summaries,img_base64,image_summaries)

In [62]:
retriever_multivector

MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018C67BA86B0>, docstore=<langchain_core.stores.InMemoryStore object at 0x0000018C677DC4D0>, search_kwargs={})

In [63]:
import re
def looks_like_base64(sb):
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$",sb ) is not None

In [64]:
import base64
import io
from PIL import Image
def resize_base64_images(base64_string,size=(128,128)):
    """
    Resize an image encoded as a base64 string
    """
    img_data=base64.b64decode(base64_string)
    img=Image.open(io.BytesIO(img_data))
    
    resized_img=img.resize(size,Image.LANCZOS)
    
    buffered=io.BytesIO()
    resized_img.save(buffered,format=img.format)
    
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [65]:
def is_image_data(b64data):
    image_signatures={
        b"\xFF\xD8\xFF":"jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",  # PNG
        b"\x47\x49\x46\x38":"gif",
        b"\x52\x49\x46\x46" : "webp",
    }
    try:
        header=base64.b64decode(b64data)[:8]
        for sig in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

In [66]:
def split_image_text_types(docs):
    """
    split base64-encoded images and texts
    """
    b64_img=[]
    texts=[]
    for doc in docs:
        if isinstance(doc,Document):
            doc=doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc=resize_base64_images(doc,size=(1300,600))
            b64_img.append(doc)
        else:
            texts.append(doc)
    print(b64_img)
    print(texts)
    return {"images":b64_img,"texts":texts}

In [75]:
def img_prompt_func(data_dict):
    """
    join the context into a single string
    """
    formatted_texts="\n".join(data_dict["content"]["texts"])
    messages=[]
    if data_dict["content"]["images"]:
        for image in data_dict["content"]["images"]:
            image_messages={
                "type":"image_url",
                    "image_url":{"url":f"data:image/jpeg;base64,{image}"},   
            }
            messages.append(image_messages)
    
    text_message={
        "type":"text",
        "text":(
            "you are a helpful assistant.\n"
            "you will be given a mixed info(s). \n"
            "use this information to provide relevant information to the user question. \n"
            f"user provided question :{data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
        
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]

In [76]:
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
def multi_model_rag_chain(retriever):
    """
    multi-modal rag chain
    """
    chain=(
        {
            "content":retriever | RunnableLambda(split_image_text_types),
            "question":RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        |model
        |StrOutputParser()
    )
    return chain

In [77]:
chain_multimodal_rag=multi_model_rag_chain(retriever_multivector)

In [70]:
chain_multimodal_rag

{
  content: MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018C67BA86B0>, docstore=<langchain_core.stores.InMemoryStore object at 0x0000018C677DC4D0>, search_kwargs={})
           | RunnableLambda(split_image_text_types),
  question: RunnablePassthrough()
}
| RunnableLambda(img_prompt_func)
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000018C64B8CDA0>, default_metadata=())
| StrOutputParser()

In [72]:
query="can you explain me this left:NQ performance as more documents are retrieved. center Retrieval recall performance\
    is NQ. Right : MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved."

In [73]:
Query1="EXplain any images / figures in the paper with left: NQ  performance as more documents are retrieval. center Retrieval recall performance\
    is NQ. Right : MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved."

In [79]:
chain_multimodal_rag.invoke(Query1)

['...']
['Effect of Retrieving more documents Models are trained with either 5 or 10 retrieved latent documents, and we do not observe signiﬁcant differences in performance between them. We have the ﬂexibility to adjust the number of retrieved documents at test time, which can affect performance and runtime. Figure 3 (left) shows that retrieving more documents at test time monotonically improves Open-domain QA results for RAG-Sequence, but performance peaks for RAG-Token at 10 retrieved documents. Figure 3 (right) shows that retrieving more documents leads to higher Rouge-L for RAG-Token at the expense of Bleu-1, but the effect is less pronounced for RAG-Sequence.', 'Single-Task Retrieval Prior work has shown that retrieval improves performance across a variety of NLP tasks when considered in isolation. Such tasks include open-domain question answering [5, 29], fact checking [56], fact completion [48], long-form question answering [12], Wikipedia article generation [36], dialogue [41, 

'Here\'s an explanation of the three figures based on the provided images and text:\n\nThe figures illustrate the impact of retrieving a varying number of documents (`K`) on different performance metrics for RAG (Retrieval-Augmented Generation) models.\n\n**Left Figure: NQ performance as more documents are retrieved (NQ Exact Match)**\n\n*   **X-axis:** K Retrieved Docs (Number of retrieved documents)\n*   **Y-axis:** NQ Exact Match (Accuracy on the Natural Questions open-domain QA dataset)\n*   **Models:** RAG-Tok (RAG-Token, solid orange line) and RAG-Seq (RAG-Sequence, dashed green line).\n\nThis graph shows how the accuracy of answering natural questions changes as more documents are retrieved during inference.\n*   **RAG-Sequence (RAG-Seq):** The dashed green line for RAG-Sequence shows a **monotonic improvement** in NQ Exact Match as the number of retrieved documents (`K`) increases. It starts around 39% for very few documents and steadily rises to over 44% for 40-50 retrieved do